In [1]:
# 라이브러리
import os
import re
import glob
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split

# 데이터 불러오기
* 노드에서 제공된 코드를 최대한 활용

In [8]:
seed = 1234
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 데이터 전처리

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [4]:
preprocessed_corpus = []
for sentence in raw_corpus:
    # 문장이 없거나
    if len(sentence) == 0: 
        continue
    # 마지막이 : 로 끝나면 반복문을 넘어감
    if sentence[-1] == ":": 
        continue
    # 문장을 전처리함수에 넣고 corpus에 추가
    preprocessed_corpus.append(preprocess_sentence(sentence))

In [5]:
# 토크나이징 함수
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=14)  

    print(tensor,'\n',tokenizer)
    return tensor, tokenizer

In [6]:
data, tokenizer = tokenize(preprocessed_corpus)

[[   2  304   28 ...    0    0    0]
 [   2  221   13 ...    0    0    0]
 [   2   24   17 ...    0    0    0]
 ...
 [   2   48   16 ...    0    0    0]
 [   9 2883   14 ...  264   19    3]
 [   2    6  179 ...    0    0    0]] 


# 데이터셋 분리

In [7]:
src_input = data[:, :-1]
tgt_input = data[:, 1:]

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=seed)

In [11]:
# shape확인
enc_train.shape, enc_val.shape, dec_train.shape, dec_val.shape

((140599, 13), (35150, 13), (140599, 13), (35150, 13))

In [10]:
# 하이퍼파라미터 셋팅
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    
# tf data 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [12]:
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(256, drop_remainder=True)

# 모델학습
* 노드에서 제공된 keras subclass 모델을 활용
* 이것 저것 좀 더 추가해보자 -> 이것 저것 추가해보는 것에 의의를 두는 것이지 성능이 무조건 더 좋아진다는 것은 아님
* Learning_rate scheduler도 한 번 이용해보자

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, recurrent_initializer='glorot_uniform')
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True, recurrent_initializer='glorot_uniform')
        self.linear = tf.keras.layers.Dense(vocab_size)
        self.batchnorm_1 = tf.keras.layers.BatchNormalization()
        self.batchnorm_2 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(0.2)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.batchnorm_1(out)
        out = self.rnn_2(out)
        out = self.batchnorm_2(out)
        out = self.dropout(out)
        out = self.linear(out)
        
        return out

In [14]:
# 에포크마다 학습률을 약 10%씩 감소시켜보자
def simple_learning_rate_decay(epoch, lr):
    return lr * np.exp(-0.1)

In [13]:
np.exp(-0.1)

0.9048374180359595

In [20]:
# 콜백함수 등 선언
# 러닝레이트 스케쥴러
lr_scheduler = LearningRateScheduler(simple_learning_rate_decay)
# 옵티마이저
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
# 로스
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
# 얼리스탑
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=7)

In [18]:
embedding_size = 512
hidden_size = 1024
model = TextGenerator(VOCAB_SIZE, embedding_size , hidden_size)

In [19]:
model.compile(loss=loss, optimizer=optimizer)

In [21]:
# 모델에 이것저것 막 추가했더니 별로군요
model.fit(train_dataset, epochs=30, validation_data=val_dataset, callbacks=[lr_scheduler, early_stopping])

Epoch 1/30
549/549 [==============================] - 118s 216ms/step - loss: 3.4772 - val_loss: 3.7988 - lr: 4.5242e-04
Epoch 2/30
549/549 [==============================] - 115s 209ms/step - loss: 2.8312 - val_loss: 2.7927 - lr: 4.0937e-04
Epoch 3/30
549/549 [==============================] - 115s 209ms/step - loss: 2.4515 - val_loss: 2.6398 - lr: 3.7041e-04
Epoch 4/30
549/549 [==============================] - 126s 229ms/step - loss: 2.1574 - val_loss: 2.5514 - lr: 3.3516e-04
Epoch 5/30
549/549 [==============================] - 120s 219ms/step - loss: 1.9315 - val_loss: 2.5017 - lr: 3.0327e-04
Epoch 6/30
549/549 [==============================] - 118s 216ms/step - loss: 1.7584 - val_loss: 2.4810 - lr: 2.7441e-04
Epoch 7/30
549/549 [==============================] - 119s 216ms/step - loss: 1.6243 - val_loss: 2.4655 - lr: 2.4829e-04
Epoch 8/30
549/549 [==============================] - 117s 213ms/step - loss: 1.5176 - val_loss: 2.4674 - lr: 2.2466e-04
Epoch 9/30
549/549 [============

# 텍스트 생성 테스트

In [22]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [23]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a monster <end> '

## 그는 괴물이야